In [ ]:
import os
import json
import pandas as pd
from datetime import datetime

# Import Google Cloud libraries
from google.cloud import storage
import vertexai
from vertexai.generative_models import GenerativeModel, Part

# --- CONFIGURATION ---
# TODO: You MUST update these four values for your project to work.



# Use a raw string (r"...") for Windows paths
LOCAL_PDF_PATH = r"C:\path\to\your\document.pdf"
OUTPUT_EXCEL_PATH = r"C:\path\to\your\output\financial_summary.xlsx"

print("Configuration loaded successfully.")

GOOGLE INIT

In [ ]:
# Initialize the client for Google Cloud Storage
storage_client = storage.Client(project=PROJECT_ID)

# Initialize the Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=LOCATION)

print("Google Cloud clients initialized successfully.")
print(f"Ready to interact with project: '{PROJECT_ID}'")

Google Cloud

In [ ]:
# Create a unique name for the object in GCS to avoid conflicts
gcs_object_name = f"temp-uploads/{os.path.basename(LOCAL_PDF_PATH)}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
gcs_uri = f"gs://{GCS_BUCKET_NAME}/{gcs_object_name}"

# Get a reference to the GCS bucket and blob (the file)
bucket = storage_client.bucket(GCS_BUCKET_NAME)
blob = bucket.blob(gcs_object_name)

print(f"Uploading local file '{LOCAL_PDF_PATH}' to '{gcs_uri}'...")

# Perform the upload
blob.upload_from_filename(LOCAL_PDF_PATH)

print("\nUpload complete!")
print("You can now see this file in your GCS bucket in the Google Cloud Console.")

In [ ]:
# pdf_file = Part.from_uri(
#     file_uri="Z:\\Maitra & Co\\Clients\\S\\Sodha Ashwin_Heena\\Ashwin Sodha\\AY 2025-26\\Income Tax\\Data from client\\Ashwin Sodha- 1_merged",
#     mime_type= "application/pdf"
# )

In [ ]:
prompt = """
Extract all transactions from the provided financial document (statement or passbook) into a raw CSV string.

**Output Schema & Headers (7 columns, exact order):**
`Date,ChequeNo,Narration,ValueDate,WithdrawalAmount,DepositAmount,ClosingBalance`

**Processing Rules:**
- **Combine Multi-line Narration:** Merge multi-line transaction descriptions (like 'Particulars' or 'Narration') into a single field with spaces.
- **Handle Missing Columns:** If a source document has no 'ValueDate', keep the column in the header but leave its data fields empty.
- **Data Cleaning:** Remove all non-numeric characters from amount columns (e.g., '₹', ',', 'Cr', 'Dr'). `1,234.56 Cr` must become `1234.56`.
- **Zero Values:** Represent empty or zero withdrawals/deposits as `0.00`.
- **CRITICAL COMMA RULE:** If any field's text contains a comma, enclose that entire field in double quotes. Example: `...,"Transfer, Savings Account",...`
- **Exclusions:** Do not include any summary or footer lines (e.g., 'Clear Balance', 'Carried Forward', 'We provide ATM Cards...').

**Final Output:**
- Raw CSV text only.
- No explanations, summaries, or markdown.
- Start directly with the header row.
"""

In [8]:
client = genai.Client(project="banktopdf", location="global", vertexai=True)
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[pdf_file, prompt],
)
print(response.text)

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Request contains an invalid argument.', 'status': 'INVALID_ARGUMENT'}}